<font size=6>Développer une preuve de concept</font>  
<font size=5>Classification d'image avec le papier Learning What and Where to Transfer  
Partie 1 : Améliorer les performances d'EfficientNetB0 grâce à EfficientNetB7</font>



---

**Vérification de l'environnement**

In [ ]:
import sys
IN_COLAB = "google.colab" in sys.modules
# PATH_DRIVE : to change according to your Google Drive folders
PATH_DRIVE = "/content/drive/My Drive/MachineLearning/ML07"
# IMAGES_DRIVE : to get access to previously loaded images
IMAGES_DRIVE = "/content/drive/My Drive/MachineLearning/ML06/Images"

In [ ]:
if IN_COLAB:
    print("Le notebook est exécuté sur Google Colab")
else:
    print("Le notebook est exécuté en local")

Le notebook est exécuté sur Google Colab


In [ ]:
if IN_COLAB:
    from google.colab import drive, files
    drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


---
## <font color=blue>Notebook set-up</font>

**Importation des librairies**

In [ ]:
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import re
import datetime
import random as python_random

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
import csv

In [ ]:
if IN_COLAB:
    sys.path.append(PATH_DRIVE)
    os.chdir(PATH_DRIVE)
    import sf_graphiques as sfg
    import sf_efficientnet_b4 as sf_efn
    import sf_pytorch_loaders as sf_load
    import sf_meta_optimizers as sf_optim
    import sf_l2t_ww as sf_l2t
else:
    import modules_perso.sf_graphiques as sfg
    import modules_perso.sf_efficientnet_b4 as sf_efn
    import modules_perso.sf_pytorch_loaders as sf_load
    import modules_perso.sf_meta_optimizers as sf_optim
    import modules_perso.sf_l2t_ww as sf_l2t

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


**Notebook set-up**

In [ ]:
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = fig_id + "." + fig_extension
    if IN_COLAB:
        path = PATH_DRIVE + "/" + path
    #print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, dpi=resolution)

In [ ]:
RANDOM_SEED = 42
BATCH_SIZE = 32

In [ ]:
def reset_random_seeds():
    np.random.seed(RANDOM_SEED)
    python_random.seed(RANDOM_SEED)
    torch.manual_seed(40)

In [ ]:
reset_random_seeds()

In [ ]:
if torch.cuda.device_count() >= 1:
    print(torch.cuda.get_device_name(0))
else:
    print("No Cuda Device")
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

Tesla P100-PCIE-16GB


In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def accuracy_topk(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(1.0 / batch_size))
    return res

In [ ]:
def accuracy_top1(outputs_data, labels):
    _, pred = torch.max(outputs_data, 1)
    correct = (pred == labels).sum().item()
    return correct / labels.size(0)

In [ ]:
def get_validation(model, loader):
    acc = AverageMeter()
    loss = AverageMeter()
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            y_pred = model.forward(x)
            loss.update(F.cross_entropy(y_pred, y))
            acc.update(accuracy_top1(y_pred.data, y))
    return loss.avg, acc.avg

In [ ]:
def transferlearning_on_targettask(mymodel, mycriterion, myoptimizer, myloaders,
                                   mysave_name, nb_epochs=50, patience=0):
    csv_filename = os.path.join(PATH_DRIVE, "logs/{}.csv".format(mysave_name))
    csv_memory = open(csv_filename, "w")  # Write the headers to the file
    writer = csv.writer(csv_memory)
    writer.writerow(["epoch", "validation_loss", "training_loss",
                      "validation_accuracy", "training_accuracy"])
    csv_memory.close()

    best_acc = 0.0
    lag_best = 0
    for epoch in range(nb_epochs):
        mymodel.train()
        epoch_acc = AverageMeter()
        epoch_loss = AverageMeter()
        for x, y in myloaders[0]:
            x, y = x.to(device), y.to(device)
            myoptimizer.zero_grad()
            y_pred = mymodel.forward(x)
            loss = mycriterion(y_pred, y)
            loss.backward()
            myoptimizer.step()
            epoch_loss.update(loss.item())
            epoch_acc.update(accuracy_top1(y_pred.data, y))
        training_loss = epoch_loss.avg
        training_accuracy = epoch_acc.avg
        
        validation_loss, validation_accuracy = get_validation(mymodel, myloaders[2])
        if validation_accuracy > best_acc:
            best_acc = validation_accuracy
            lag_best = 0
            torch.save(mymodel.state_dict(), "models/{}.pth".format(mysave_name))
        else:
            lag_best += 1

        print("[Epoch : {}]  [Training accuracy = {:.2%}] [Validation accuracy = {:.2%}] [Best = {:.2%}]  [Training loss = {:.3f}] [Validation loss = {:.3f}]"\
              .format(epoch, training_accuracy, validation_accuracy, best_acc, training_loss, validation_loss))
        csv_memory = open(csv_filename, "a")  # Write the results to the file
        writer = csv.writer(csv_memory)
        writer.writerow([epoch, validation_loss, training_loss,
                        validation_accuracy, training_accuracy])
        csv_memory.close()

        if (patience > 0) & (lag_best > patience):
            print("Learning ended by early-stop")
            break
        # next epoch

---
## 1\. Performance de base d'EfficientNetB0

### 1.1. Optimisation avec l'algorithme Adam

In [ ]:
model = sf_efn.efficientnetb0(num_classes=120, pretrained=True,
                              url_pretrained="models/efficientnet-b0-355c32eb.pth")
model.freeze_layers(last_layer=-2)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), eps=1e-7)
# note : eps=1e-7 to get the same epsilon than Tensorflow

0 : _conv_stem
freezed
1 : _bn0
freezed
2 : _blocks
freezed
3 : _conv_head
freezed
4 : _bn1
freezed
5 : _avg_pooling
freezed
6 : _dropout
freezed
7 : _fc
unfreezed
8 : _swish
unfreezed


In [ ]:
loaders = sf_load.get_dataset(IMAGES_DRIVE, model, mini_data=0.1, stratify=True,
                              batch_size=64, with_data_augmentation=False)

In [ ]:
transferlearning_on_targettask(model, criterion, optimizer, loaders, "efnb0_120_adam")

[Epoch : 0]  [Training accuracy = 7.77%] [Validation accuracy = 34.74%] [Best = 34.74%]  [Training loss = 4.615] [Validation loss = 4.108]
[Epoch : 1]  [Training accuracy = 61.97%] [Validation accuracy = 50.29%] [Best = 50.29%]  [Training loss = 3.638] [Validation loss = 3.446]
[Epoch : 2]  [Training accuracy = 83.78%] [Validation accuracy = 54.29%] [Best = 54.29%]  [Training loss = 2.901] [Validation loss = 2.918]
[Epoch : 3]  [Training accuracy = 91.17%] [Validation accuracy = 56.78%] [Best = 56.78%]  [Training loss = 2.303] [Validation loss = 2.527]
[Epoch : 4]  [Training accuracy = 93.97%] [Validation accuracy = 57.89%] [Best = 57.89%]  [Training loss = 1.826] [Validation loss = 2.260]
[Epoch : 5]  [Training accuracy = 96.65%] [Validation accuracy = 57.88%] [Best = 57.89%]  [Training loss = 1.477] [Validation loss = 2.087]
[Epoch : 6]  [Training accuracy = 97.84%] [Validation accuracy = 57.51%] [Best = 57.89%]  [Training loss = 1.191] [Validation loss = 1.981]
[Epoch : 7]  [Trainin

### 1.2. Optimisation avec l'algorithme SGD-Nesterov

In [ ]:
model = sf_efn.efficientnetb0(num_classes=120, pretrained=True,
                              url_pretrained="models/efficientnet-b0-355c32eb.pth")
model.freeze_layers(last_layer=-2)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=True)

0 : _conv_stem
freezed
1 : _bn0
freezed
2 : _blocks
freezed
3 : _conv_head
freezed
4 : _bn1
freezed
5 : _avg_pooling
freezed
6 : _dropout
freezed
7 : _fc
unfreezed
8 : _swish
unfreezed


In [ ]:
transferlearning_on_targettask(model, criterion, optimizer, loaders,
                               "efnb0_120_sgd", nb_epochs=200, patience=20)

[Epoch : 0]  [Training accuracy = 1.86%] [Validation accuracy = 10.30%] [Best = 10.30%]  [Training loss = 4.748] [Validation loss = 4.594]
[Epoch : 1]  [Training accuracy = 19.54%] [Validation accuracy = 32.77%] [Best = 32.77%]  [Training loss = 4.435] [Validation loss = 4.289]
[Epoch : 2]  [Training accuracy = 44.72%] [Validation accuracy = 41.46%] [Best = 41.46%]  [Training loss = 4.097] [Validation loss = 3.993]
[Epoch : 3]  [Training accuracy = 59.12%] [Validation accuracy = 45.35%] [Best = 45.35%]  [Training loss = 3.767] [Validation loss = 3.723]
[Epoch : 4]  [Training accuracy = 67.65%] [Validation accuracy = 47.00%] [Best = 47.00%]  [Training loss = 3.456] [Validation loss = 3.486]
[Epoch : 5]  [Training accuracy = 71.65%] [Validation accuracy = 48.75%] [Best = 48.75%]  [Training loss = 3.191] [Validation loss = 3.283]
[Epoch : 6]  [Training accuracy = 77.80%] [Validation accuracy = 49.11%] [Best = 49.11%]  [Training loss = 2.928] [Validation loss = 3.114]
[Epoch : 7]  [Trainin

---
## 2\. Transfert de connaissance d'EfficientNetB7 vers EfficientNetB0

In [ ]:
model_efficientnetb7_url = PATH_DRIVE + "/models/efficientnet-b7-dcc49843.pth"

In [ ]:
source_model = sf_efn.efficientnetb7(pretrained=True, max_num_features=512,
                                     folder_saves="models",
                                     url_pretrained=model_efficientnetb7_url)
target_model = sf_efn.efficientnetb0(num_classes=120, pretrained=False)
target_model.load_state_dict(torch.load("models/efnb0_120_adam.pth"))

<All keys matched successfully>

In [ ]:
loaders = sf_load.get_dataset(IMAGES_DRIVE, target_model, mini_data=0.1, stratify=True,
                              batch_size=64, with_data_augmentation=False)
model = target_model.to(device)
get_validation(model, loaders[2])

(tensor(2.2600, device='cuda:0'), 0.5788976648351649)

In [ ]:
l2t_model = sf_l2t.WhatWhere(source_model, target_model, device=device)

In [ ]:
l2t_model.train(IMAGES_DRIVE, epochs=200, batch_size=8,
                mini_data=0.1, with_data_augmentation=False,
                early_stop=True, save_name="efnb7_to_efnb0_mini")

[2020-11-17 12:28:52,155] [main] /usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-7cfdf943-b60f-4a16-aee8-db472feec61a.json


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


[2020-11-17 12:50:40,277] [main] Best model is saved
[2020-11-17 12:50:40,608] [main] Last model is saved
[2020-11-17 12:50:40,610] [main] [Epoch : 1]  [Training accuracy = 0.76%] [Validation accuracy = 0.73%] [Best = 0.73%]  [Training loss = nan] [Validation loss = nan]


La méthode retourne une loss (la cross-entropy) égale à NA. Cela est dû à l'instabilité des gradients qui fait que la perte explose et n'est plus calculable.  
Cela remet en cause la possibilité d'utiliser cette méthode sur toutes sortes de réseaux, même lorsque ceux-ci utilisent des couches de Batch Normalization.